In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense
from tensorflow.keras.layers import Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.regularizers import l2

2024-02-06 06:14:17.849911: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-06 06:14:17.850033: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-06 06:14:18.003969: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
from tensorflow.keras import backend as K

def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2 * (precision * recall) / (precision + recall + K.epsilon())
    return f1_val

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Data Preparation

batch_size = 32

datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    '/kaggle/input/covidqu/Infection Segmentation Data/Infection Segmentation Data/Train',
    target_size=(256, 256),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    '/kaggle/input/covidqu/Infection Segmentation Data/Infection Segmentation Data/Val',
    target_size=(256, 256),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)



Found 8948 images belonging to 3 classes.
Found 557 images belonging to 3 classes.


In [5]:


# Directory where the checkpoints will be saved
checkpoint_dir = 'Custom_Model/'
os.makedirs(checkpoint_dir, exist_ok=True)

# Define a ModelCheckpoint callback to save the model at every improved epoch
checkpoint = ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, 'model_epoch_{epoch:02d}_train_acc_{accuracy:.4f}_val_acc_{val_accuracy:.4f}.h5'),
    monitor='val_accuracy',  # metric to monitor
    verbose=1,  # verbosity - 0 or 1
    save_best_only=True,  # Save only the best model according to the monitored metric
    mode='max',  # Since we are monitoring accuracy, higher is better
    save_weights_only=False  # Save the full model, not just the weights
)

# Create a custom CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(256, 256, 3), activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (1, 1), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu', name='fc1'))
model.add(Dense(32, activation='relu', name='fc2'))
model.add(Dense(3, activation='softmax'))  # Assuming 3 classes

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', f1_score])

# Adjust the `workers` parameter as necessary
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=50,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks=[checkpoint],
    use_multiprocessing=True,  # Enable multiprocessing
    workers=4  # Number of workers to use. Adjust as per your CPU cores.
)


Epoch 1/50


I0000 00:00:1707200086.381884      88 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


280/280 [==============================] - ETA: 0s - loss: 0.8286 - accuracy: 0.5971 - f1_score: 0.5080
Epoch 1: val_accuracy improved from -inf to 0.55655, saving model to Custom_Model/model_epoch_01_train_acc_0.5971_val_acc_0.5566.h5


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


280/280 [==============================] - 117s 349ms/step - loss: 0.8286 - accuracy: 0.5971 - f1_score: 0.5080 - val_loss: 1.0770 - val_accuracy: 0.5566 - val_f1_score: 0.0199
Epoch 2/50
280/280 [==============================] - ETA: 0s - loss: 0.7062 - accuracy: 0.6586 - f1_score: 0.6236
Epoch 2: val_accuracy improved from 0.55655 to 0.60503, saving model to Custom_Model/model_epoch_02_train_acc_0.6586_val_acc_0.6050.h5
280/280 [==============================] - 92s 323ms/step - loss: 0.7062 - accuracy: 0.6586 - f1_score: 0.6236 - val_loss: 0.9708 - val_accuracy: 0.6050 - val_f1_score: 0.1931
Epoch 3/50
280/280 [==============================] - ETA: 0s - loss: 0.6568 - accuracy: 0.6750 - f1_score: 0.6541
Epoch 3: val_accuracy improved from 0.60503 to 0.61041, saving model to Custom_Model/model_epoch_03_train_acc_0.6750_val_acc_0.6104.h5
280/280 [==============================] - 91s 320ms/step - loss: 0.6568 - accuracy: 0.6750 - f1_score: 0.6541 - val_loss: 0.8834 - val_accuracy: 0